In [2]:
def No_Unique_Solution(A,B):
    N=len(A[0])     # Number of Variables
    M=len(A)        # Number of Equations
    counter=0
    for row in range(M-1,-1,-1):
        check=False
        for col in range(N):
            if(abs(A[row][col])>2e-19):
                check=True
        if(check==False):
            counter+=1
            if(abs(B[row])>2e-19):
                return "No Solution"
    if(counter<M-N):
        return "No Solution"
    elif(counter>M-N):
        return "Infinite Solution"
    else:
        return "Unique Solution"
    

In [3]:
def Forward_Elimination(A,B):
    N=len(A[0])     # Number of Variables
    M=len(A)        # Number of Equations
    flag="Unique Solution"
    for row in range(0,N):
        #Check if Normalization Possible
        if(abs(A[row][row])<=2e-19):
            #Find where its non-0
            swapped=False
            for dummy_row in range(row+1,M):
                if(abs(A[dummy_row][row])>2e-19):
                    #Swap
                    A[dummy_row],A[row]=A[row],A[dummy_row]
                    B[dummy_row],B[row]=B[row],B[dummy_row]
                    swapped=True
                    break
            if(swapped==False):
                #No Unique Solution
                flag=No_Unique_Solution(A,B)
                return A,B,flag
        divisor=A[row][row]
        #Normalization
        for col in range(row,N):
            A[row][col]/=divisor
        B[row]/=divisor
        #Elimination
        for next_rows in range(row+1,M):
            multiplier=A[next_rows][row]
            for col in range(row,N):
                A[next_rows][col]-=multiplier*A[row][col]
            B[next_rows]-=multiplier*B[row]
    flag=No_Unique_Solution(A,B)
    return A,B,flag

In [4]:
def Backward_Substitution(A, B):
    N=len(A[0])     # Number of Variables
    M=len(A)        # Number of Equations
    #Create the list x containing the values of the variables
    x=['none' for i in range(N)]
    for row in range(N-1,-1,-1):
        Sum=B[row]
        for cols in range(N-1,row,-1):
            Sum-=x[cols]*A[row][cols]
        x[row]=Sum
    return x

In [5]:
import numpy as np
A=[
    [1,2,3],
    [8,9,10],
    [33,38,43]
]
B=[1,5,21]

def Gauss_Elimination(A,B):
    if(not isinstance(A,list)):
        A=A.astype(np.float32)
        B=B.astype(np.float32)
    A1,B1,flag=Forward_Elimination(A,B)
    if(flag=="Unique Solution"):
        x=Backward_Substitution(A1,B1)
        return x
    else:
        print(flag)
Gauss_Elimination(A,B)


Infinite Solution


In [21]:
def circuit_solver(filename):
    with open(filename) as f:
        lines=f.readlines()
        ckt_data=[]
        for line in lines:
            ckt_data.append(line.split())
    check=False
    variables=[]
    dc_flag=False
    ac_flag=False
    for line in ckt_data:
        if(len(line)==0):
            continue
        if(line[0]=='.end'):
            check=False
            break
        if(check==True):
            if(line[1][0]!='n' and line[1]!='GND'):
               line[1]='n'+line[1]
            if(line[2][0]!='n' and line[2]!='GND'):
                line[2]='n'+line[2]
            variables.append(line[1])        
            variables.append(line[2])    
            if(line[0][0] in ['V','v']):
                variables.append("I"+line[0])  
                if(line[3]=="dc"):
                    dc_flag=True
                if(line[3]=="ac"):
                    ac_flag=True
            elif(line[0][0] in ['I','i']):
                variables.append(line[0])
                if(line[3]=="dc"):
                    dc_flag=True
                if(line[3]=="ac"):
                    ac_flag=True
        if(line[0]=='.circuit'):
            check=True
    variables=list(set(variables))
    pos={}
    for i in range(len(variables)):
        pos[variables[i]]=i
    if(ac_flag==True and dc_flag==True):
        print("Multiple Frequencies : Involves both AC and DC Sources")
        return
    elif(dc_flag==True):
        dc_solver(pos,ckt_data,variables)
    else:
        ac_solver(pos,ckt_data,variables)

def print_MNA(x,variables):
    for i in range(len(variables)):
        if(variables[i][0]=='I'):
            print(variables[i]," = ",x[i]," A",sep="")
        else:
            print(variables[i]," = ",x[i]," V",sep="")

def dc_solver(pos,ckt_data,variables):
    l=len(variables)
    A=[[0 for i in range(l)] for j in range(l)]
    b=[0 for i in range(l)]
    check=False
    sources=['V','v','I','i']
    for line in ckt_data:
        if(line[0]=='.end'):
            check=False
            break
        if(check==True):
            if(line[0][0] not in sources):
                impedance=float(line[3])
                A[pos[line[1]]][pos[line[1]]]+=1/impedance
                A[pos[line[1]]][pos[line[2]]]-=1/impedance
                A[pos[line[2]]][pos[line[2]]]+=1/impedance
                A[pos[line[2]]][pos[line[1]]]-=1/impedance
            elif(line[0][0] in sources):
                A[pos["I"+line[0]]][pos[line[2]]]=1
                A[pos["I"+line[0]]][pos[line[1]]]=-1
                b[pos["I"+line[0]]]=float(line[4])
                A[pos[line[2]]][pos["I"+line[0]]]=1
                A[pos[line[1]]][pos["I"+line[0]]]=-1
            else:
                print(".netlist input Error")
                return
        if(line[0]=='.circuit'):
            check=True
    #Removing Ground Node to get the Official Form of MNA
    A.pop(pos["GND"])
    b.pop(pos["GND"])
    for row in A:
        row.pop(pos["GND"])
    variables.pop(pos["GND"])
    for key in pos:
        if(pos[key]>pos["GND"]):
            pos[key]-=1
    del pos["GND"]
    #It has been reduced to the MNA form
    # print(A)
    # print(b)
    x=Gauss_Elimination(A,b)
    print_MNA(x,variables)

circuit_solver("ckt1.netlist")
print()
circuit_solver("ckt3.netlist")

n3 = 0.0 V
n4 = 5.0 V
IV1 = -0.0005 A
n1 = 0.0 V
n2 = 0.0 V

n3 = 2.573099415204678 V
n4 = 1.4035087719298247 V
n5 = 0.9356725146198832 V
IV1 = -0.004970760233918129 A
n1 = 10.0 V
n2 = 5.029239766081871 V


In [22]:
import math
def ac_solver(pos,ckt_data,variables):
    #read frequency
    counter=0
    for i in range(len(ckt_data)-1,0,-1):
        if(len(ckt_data[i])==0):
            continue
        if(ckt_data[i][0]=='.ac'):
            counter+=1
            frequency=float(ckt_data[i][2])
    if(counter!=1):
        print("Multiple frequencies")
        return
    omega=2*math.pi*frequency
    print("got omega", omega)
    l=len(variables)
    A=[[0 for i in range(l)] for j in range(l)]
    b=[0 for i in range(l)]
    check=False
    sources=['V','v','I','i']
    im_j=complex(0+1j)
    for line in ckt_data:
        if(len(line)==0):
            continue
        if(line[0]=='.end'):
            check=False
            break
        if(check==True):
            if(line[0][0] not in sources):
                impedance=float(line[3])
                if(line[0][0] in ['C','c']):
                    impedance=1/(im_j*impedance*omega)
                if(line[0][0] in ['L','l']):
                    impedance=im_j*impedance*omega
                print("impedance of ",line[0],impedance)
                A[pos[line[1]]][pos[line[1]]]+=1/impedance
                A[pos[line[1]]][pos[line[2]]]-=1/impedance
                A[pos[line[2]]][pos[line[2]]]+=1/impedance
                A[pos[line[2]]][pos[line[1]]]-=1/impedance
            elif(line[0][0] in ['V','v']):
                A[pos["I"+line[0]]][pos[line[2]]]=1
                A[pos["I"+line[0]]][pos[line[1]]]=-1
                b[pos["I"+line[0]]]=float(line[4])+float(line[5])*im_j
                A[pos[line[2]]][pos["I"+line[0]]]=1
                A[pos[line[1]]][pos["I"+line[0]]]=-1
            elif(line[0][0] in ['I','i']):
                A[pos[line[0]]][pos[line[0]]]=1
                b[pos[line[0]]]=float(line[4])+float(line[5])*im_j
                A[pos[line[1]]][pos[line[0]]]=-1
                A[pos[line[2]]][pos[line[0]]]=1
            else:
                print(".netlist input Error")
                return
        if(line[0]=='.circuit'):
            check=True
    A.pop(pos["GND"])
    b.pop(pos["GND"])
    for row in A:
        row.pop(pos["GND"])
    variables.pop(pos["GND"])
    for key in pos:
        if(pos[key]>pos["GND"]):
            pos[key]-=1
    del pos["GND"]    
    print(A)
    print(b)
    print(variables)
    print(pos)
    x=Gauss_Elimination(A,b)
    print_MNA(x,variables)
circuit_solver("ckt7.netlist")

got omega 6283.185307179586
impedance of  C1 -0.00015915494309189535j
impedance of  R1 1000.0
impedance of  L1 0.006283185307179586j
[[1, 0], [1, (0.001+6124.03036408769j)]]
[(5+0j), 0]
['I1', 'n1']
{'I1': 0, 'n1': 1}
I1 = (5+0j) A
n1 = (-1.3332000879741771e-10+0.000816455782015824j) V


In [1]:
import cmath
def phasor(value):
    return abs(value),cmath.phase(value)

phasor(1+1j) 

(1.4142135623730951, 0.7853981633974483)

In [29]:
# circuit_solver("ckt1.netlist")
# circuit_solver("ckt2.netlist")
# circuit_solver("ckt3.netlist")
# circuit_solver("ckt4.netlist")
# circuit_solver("ckt5.netlist")
# circuit_solver("ckt6.netlist")
# circuit_solver("ckt7.netlist")

got omega 6283.185307179586
impedance of  C1 -0.00015915494309189535j
impedance of  R1 1000.0
impedance of  L1 0.006283185307179586j
[[1, 0], [1, (0.001+6124.03036408769j)]]
[(5+0j), 0]
['I1', 'n1']
{'I1': 0, 'n1': 1}
I1 = (5+0j) A
n1 = (-1.3332000879741771e-10+0.000816455782015824j) V


In [4]:
a=3+0j
print(a)
print(type(a))

(3+0j)
<class 'complex'>


In [2]:
abs(a)

5.0

In [5]:
import cmath
cmath.phase(a)

0.0